In [2]:
import pandas as pd 
import numpy as np
import math

data = pd.read_csv("sg_covid_data.csv")

data = data.drop(columns=['False Positives Found'])

### 1. Profile the original dataset, by getting the following properties of each column

In [3]:
# Data Type
data_types = data.dtypes.to_frame(name="Data Types")

# Record Count
record_count = data.count().to_frame(name="Record Count")

# Minimum Value
min_value = data.min().to_frame(name="Minimum Value")

# Maximum Value
max_value = data.max().to_frame(name="Maximum Value")

# Mean
mean_data = data.mean().to_frame(name="Mean")

# Mode
mode_data = data.mode(dropna=True).head(1)
mode_data = mode_data.rename(index={0:'Mode'}, inplace=False)
mode_data = mode_data.transpose()

#Frequency of Mode
freq_mode = pd.DataFrame({'Frequency of Mode':[data[x].isin(data[x].mode()).sum() for x in data]}, index=data.columns)


# Merge all data to one dataframe
original_profile = pd.concat([data_types, record_count, min_value, max_value, mean_data, mode_data, freq_mode], axis=1)

# Created a function that will get a random date for mode and get the frequency of it
def changeMode(profile):
    getDateMode = data['Date'].value_counts(dropna=True).reset_index().drop(columns=['Date']).head(1)
    new_mode = getDateMode.iloc[0]['index']
    
    freq_mode_count = 0
    
    for date in data['Date']:
        if date == new_mode:
            freq_mode_count += 1 
        
    profile.iloc[0, profile.columns.get_loc('Mode')] = new_mode
    profile.iloc[0, profile.columns.get_loc('Frequency of Mode')] = freq_mode_count
    
    return profile

changeMode(original_profile)

,Mode
Date,1 02 2020
Daily Confirmed,3
Cumulative Confirmed,18
Daily Discharged,0
Passed but not due to COVID,0
Cumulative Discharged,0
Discharged to Isolation,0
Still Hospitalised,13
Daily Deaths,0
Cumulative Deaths,27


### 2. There are a few columns with persistent data issues, you are to locate each of these columns and identify their issues based on the following data quality dimensions

#### List of Validity Issues

* Letters, on column **Cumulative Confirmed** values like *[A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P]* are present

* High values, on column **Cumulative Discharged** values like *[13213123, 54353453, 23423423, 84849337, 10023233, 32432242* are present

* Negative values, on column **Still Hospitalised** values like *[-2, -1, -5]* are present

* High values, on column **Cumulative Deaths** values like *[400000]* are present

* negative values, on column **Total Completed Isolation MOH report** values like *[-32540, -33882, -34283, -55301, -55326, -55327, -55331, -55335]* are present

In [31]:
# Get the count of null values each columns
completeness = data.isnull().sum().to_frame(name='COMPLETENESS')

# COLUMNS WITH INVALID VALUES
INVALID_COL = ['Cumulative Confirmed', 'Cumulative Discharged', 'Still Hospitalised', 'Cumulative Deaths', 'Total Completed Isolation MOH report']

invalid_val_count = [0, 0, 0, 0, 0]

# Get the count of each invalid values
for x in range(1, len(data)):
    if not isinstance(data.iloc[x][INVALID_COL[0]], float):
        if str(data.iloc[x][INVALID_COL[0]]).isalpha() == True:
             invalid_val_count[0] += 1

    #if not math.isnan(float(data.iloc[x][INVALID_COL[1]])):
    if data.iloc[x][INVALID_COL[1]] != data.iloc[x-1][INVALID_COL[1]] + data.iloc[x]['Daily Discharged']:
        if not data.iloc[x][INVALID_COL[1]] != data.iloc[x+1][INVALID_COL[1]]  - data.iloc[x+1]['Daily Discharged']:
            invalid_val_count[1] += 1
    
    if int(data.iloc[x][INVALID_COL[2]]) < 0:
        invalid_val_count[2] += 1
        
    if  data.iloc[x][INVALID_COL[3]] != (int(data.iloc[x-1][INVALID_COL[3]]) + int(data.iloc[x]['Daily Deaths'])) and not data.iloc[x+1][INVALID_COL[3]] != (int(data.iloc[x-1][INVALID_COL[3]]) + int(data.iloc[x]['Daily Deaths'])):
        invalid_val_count[3] += 1

    if  data.iloc[x][INVALID_COL[4]] < 0:
        invalid_val_count[4] += 1

validity = pd.DataFrame({'VALIDITY': invalid_val_count}, columns = ['VALIDITY'], index=INVALID_COL)

# Merge COMPLETENESS and VALIDITY in one dataframe
data_issues = pd.concat([completeness, validity], axis=1)
data_issues = data_issues.fillna(0)
data_issues = data_issues.astype(int)

data_issues

,COMPLETENESS,VALIDITY
Date,0,0
Daily Confirmed,0,0
Cumulative Confirmed,20,16
Daily Discharged,0,0
Passed but not due to COVID,0,0
Cumulative Discharged,16,11
Discharged to Isolation,0,0
Still Hospitalised,0,3
Daily Deaths,0,0
Cumulative Deaths,0,1


### 3. Correct the values in the columns identified in item 2

In [32]:
updated_data = data.fillna(0)

# Correct invalid values in Cumulative Confirmed column (column has characters)
for x in range(1, len(updated_data)):
    if not isinstance(updated_data.iloc[x][INVALID_COL[0]], float):
        if str(updated_data.iloc[x][INVALID_COL[0]]).isalpha() == True:
            updated_data.iloc[x, updated_data.columns.get_loc(INVALID_COL[0])] = (int(updated_data.iloc[x-1][INVALID_COL[0]]) + int(updated_data.iloc[x]["Daily Confirmed "]))

# Correct invalues in Cumulative Discharge (current value not equals to sum of (past value + daily discharge))
    if  int(updated_data.iloc[x][INVALID_COL[1]]) != (int(updated_data.iloc[x-1][INVALID_COL[1]]) + int(updated_data.iloc[x]['Daily Discharged'])):
        updated_data.iloc[x, updated_data.columns.get_loc(INVALID_COL[1])] = (int(updated_data.iloc[x-1][INVALID_COL[1]]) + int(updated_data.iloc[x]["Daily Discharged"]))

# Correct invalid values in Still Hospitalised (negative numbers)
    if int(updated_data.iloc[x][INVALID_COL[2]]) < 0:
        #print(data.iloc[x][INVALID_COL[2]])
        updated_data.iloc[x, updated_data.columns.get_loc(INVALID_COL[2])] = (int(updated_data.iloc[x][INVALID_COL[0]]) - int(updated_data.iloc[x][INVALID_COL[1]]))

# Correct invalid values in Cumulative Deaths (current value not equals to sum of (past value + daily deaths))
    if  int(updated_data.iloc[x][INVALID_COL[3]]) != (int(updated_data.iloc[x-1][INVALID_COL[3]]) + int(updated_data.iloc[x]['Daily Deaths'])):
        updated_data.iloc[x, updated_data.columns.get_loc(INVALID_COL[3])] = (int(updated_data.iloc[x-1][INVALID_COL[3]]) + int(updated_data.iloc[x]["Daily Deaths"]))

# Convert invalid values in Total Completed Isolation MOH report (negative numbers)
    if  int(updated_data.iloc[x][INVALID_COL[4]]) < 0:
        updated_data.iloc[x, updated_data.columns.get_loc(INVALID_COL[4])] = int(abs(updated_data.iloc[x][INVALID_COL[4]]))

updated_data

,Date,Daily Confirmed,Cumulative Confirmed,Daily Discharged,Passed but not due to COVID,Cumulative Discharged,Discharged to Isolation,Still Hospitalised,Daily Deaths,Cumulative Deaths,Tested positive demise,Daily Imported,Daily Local transmission,Local cases residing in dorms MOH report,Local cases not residing in doms MOH report,Intensive Care Unit (ICU),General Wards MOH report,In Isolation MOH report,Total Completed Isolation MOH report,Total Hospital Discharged MOH report
0,23 01 2020,1,1,0,0,0.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,24 01 2020,2,3,0,0,0.0,0,3,0,0,0,2,0,0,0,0,0,0,0,0
2,25 01 2020,1,4,0,0,0.0,0,4,0,0,0,1,0,0,0,0,0,0,0,0
3,26 01 2020,0,4,0,0,0.0,0,4,0,0,0,0,0,0,0,0,0,0,0,0
4,27 01 2020,1,5,0,0,0.0,0,5,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,31 10 2020,12,58015,4,0,57935.0,32,12,0,28,0,12,0,0,0,0,42,32,55327,2586
283,1 11 2020,4,58019,11,0,57946.0,21,16,0,28,0,4,0,0,0,0,46,21,55331,2593
284,2 11 2020,1,58020,0,0,57946.0,22,16,0,28,0,1,0,0,0,0,46,22,55331,2593
285,3 11 2020,9,58029,13,0,57959.0,26,8,0,28,0,7,2,2,0,0,38,26,55334,2603


### 4. Profile the updated dataset, with similar properties assigned in item 1

In [33]:
# Updated Data Type
updated_data_types = updated_data.dtypes.to_frame(name="Data Types")

# Updated Record Count
updated_record_count = updated_data.count().to_frame(name="Record Count")

# Updated Minimum Value
updated_min_value = updated_data.min().to_frame(name="Minimum Value")

# Updated Maximum Value
updated_max_value = updated_data.max().to_frame(name="Maximum Value")

# Updated Mean
updated_mean_data = updated_data.mean().to_frame(name="Mean")

# Updated Mode
updated_mode_data = updated_data.mode(dropna=True).head(1)
updated_mode_data = updated_mode_data.rename(index={0:'Mode'}, inplace=False)
updated_mode_data = updated_mode_data.transpose()

# Updated Frequency of Mode
update_freq_mode = pd.DataFrame({'Frequency of Mode':[data[x].isin(updated_data[x].mode()).sum() for x in updated_data]}, index=data.columns)

# Merge all data to one dataframe
updated_profile = pd.concat([updated_data_types, updated_record_count, updated_min_value, updated_max_value, updated_mean_data, updated_mode_data, update_freq_mode], axis=1)

changeMode(updated_profile)

,Data Types,Record Count,Minimum Value,Maximum Value,Mean,Mode,Frequency of Mode
Date,object,287,1 02 2020,9 10 2020,NaN,14 05 2020,1
Daily Confirmed,int64,287,0,1426,202.494774,3,13
Cumulative Confirmed,object,287,NaN,NaN,NaN,0,0
Daily Discharged,int64,287,0,1337,201.951220,0,23
Passed but not due to COVID,int64,287,0,1,0.020906,0,281
Cumulative Discharged,float64,287,0,57960,27485.783972,0,12
Discharged to Isolation,int64,287,0,19667,4132.156794,0,62
Still Hospitalised,int64,287,0,4229,340.167247,13,7
Daily Deaths,int64,287,0,2,0.097561,0,262
Cumulative Deaths,int64,287,0,28,17.996516,27,90
